## The module "citipy" did not find a city near some of my coordinates and instead listed a province name under city.
### This is why there is no hotel listed for one of my results, and I removed it from the list that is used for markers

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

##Store Part I results into DataFrame
##Load the csv exported in Part I to a DataFrame

In [3]:
weather_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

In [4]:
#The country code of Namibia is NA, which is changed (during the csv to panda data frame)to NaN and messes things up
#This code changes it back
weather_data_df = weather_data_df.fillna('NA')

In [5]:
#This is to check if the NA, NaN, Namibia issue is addressed
is_NaN = weather_data_df. isnull()
row_has_NaN = is_NaN. any(axis=1)
rows_with_NaN = weather_data_df[row_has_NaN]
print(rows_with_NaN)

Empty DataFrame
Columns: [Unnamed: 0, City, Cloudiness, Country, Date, Humidity, Latitude, Longitude, Maximum Temperature, Wind Speed]
Index: []


In [6]:
#Check - Before additional cleaning
weather_data_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum Temperature,Wind Speed
0,0,Chui,4,UY,1587327011,78,-33.70,-53.46,67.73,8.21
1,1,Ponta do Sol,26,PT,1587326994,54,32.67,-17.10,58.55,8.01
2,2,Stornoway,0,GB,1587327034,70,58.21,-6.39,48.11,4.70
3,3,Atuona,48,PF,1587326972,74,-9.80,-139.03,82.96,16.98
4,4,Langerwehe,17,DE,1587327037,63,50.82,6.35,54.32,1.99
...,...,...,...,...,...,...,...,...,...,...
566,566,Manta,20,EC,1587327668,58,-0.95,-80.73,84.20,10.29
567,567,Oyama,100,JP,1587327393,75,36.30,139.80,52.36,5.46
568,568,Avera,40,US,1587327691,90,33.19,-82.53,55.87,11.41
569,569,Līsakovsk,100,KZ,1587327692,71,52.54,62.49,45.27,6.20


In [7]:
#The code here is for getting rid of the extra column and dropping incomplete rows
#change - additional cleaning
weather_data_df.dropna(inplace = True)
del weather_data_df["Unnamed: 0"]

In [8]:
#After to check to see if additional cleaning had an effect
weather_data_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum Temperature,Wind Speed
0,Chui,4,UY,1587327011,78,-33.70,-53.46,67.73,8.21
1,Ponta do Sol,26,PT,1587326994,54,32.67,-17.10,58.55,8.01
2,Stornoway,0,GB,1587327034,70,58.21,-6.39,48.11,4.70
3,Atuona,48,PF,1587326972,74,-9.80,-139.03,82.96,16.98
4,Langerwehe,17,DE,1587327037,63,50.82,6.35,54.32,1.99
...,...,...,...,...,...,...,...,...,...
566,Manta,20,EC,1587327668,58,-0.95,-80.73,84.20,10.29
567,Oyama,100,JP,1587327393,75,36.30,139.80,52.36,5.46
568,Avera,40,US,1587327691,90,33.19,-82.53,55.87,11.41
569,Līsakovsk,100,KZ,1587327692,71,52.54,62.49,45.27,6.20


## Humidity Heatmap
Configure gmaps.
Use the Lat and Lng as locations and Humidity as the weight.
Add Heatmap layer to map.
# Create a heat map that displays the humidity for every city from the part I of the homework.

In [9]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [10]:
# Store latitude and longitude in locations
locations = weather_data_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
heat_map_weight =  weather_data_df["Humidity"].astype(float)

In [11]:
# Plot Heatmap
fig = gmaps.figure(center=(0, 0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=heat_map_weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
Narrow down the cities to fit weather conditions.
Drop any rows will null values.

### * Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [12]:
ideal_waether_cities_df=weather_data_df.copy()
ideal_waether_cities_df = ideal_waether_cities_df.loc[(ideal_waether_cities_df["Maximum Temperature"] >70) & (ideal_waether_cities_df["Maximum Temperature"] <80),:]
ideal_waether_cities_df = ideal_waether_cities_df.loc[(ideal_waether_cities_df["Wind Speed"] <10),:]
ideal_waether_cities_df = ideal_waether_cities_df.loc[(ideal_waether_cities_df["Cloudiness"] ==0),:]

In [13]:
#Only threes cities met the conditions required
ideal_waether_cities_df.head(20)

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum Temperature,Wind Speed
180,Adrar,0,MR,1587327241,16,20.50,-10.07,79.21,7.43
329,Morondava,0,MG,1587327415,73,-20.28,44.28,78.60,7.45
409,San Luis,0,AR,1587327333,29,-33.30,-66.34,76.96,6.17


### Hotel Map
Store into variable named hotel_df.
Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.
Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.
Plot markers on top of the heatmap.
* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

In [14]:
hotel_df = ideal_waether_cities_df
hotel_df["Hotel Name"]=""

In [15]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum Temperature,Wind Speed,Hotel Name
180,Adrar,0,MR,1587327241,16,20.50,-10.07,79.21,7.43,
329,Morondava,0,MG,1587327415,73,-20.28,44.28,78.60,7.45,
409,San Luis,0,AR,1587327333,29,-33.30,-66.34,76.96,6.17,


In [16]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify HOTELS
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Search with params above and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAPcgKQETgDbon7DmUs6H5L8vGpZsX6DEsrGIILrBRB_N8pG2xkl-qE6DJujzd-W1bHji4roN5LXroUJwUBtsVLW74qAKW17WqO9gV9m8CT-YYS2NUzrveBtxIIdHzM-MKjuv8ZLp07HHMiH6RQUo3UM54STdM68k5Oz4RgAOYlmQH4XAkU1S32nu08BPMqv9x8ozs8ct6zVkwOpetqe5wC15rnWHolgjJcNR_OZUmMqqhxQGuhPnmeUdEp1IHlMMr3_Z2gBAM5E_m_kHO2pXYF5NGKXm5l3_OCJxgvrvWY7MXIMORnURzlwjN8ZQ94YCrRdY00EKk7IlNSuMW3UHY-QhkwoyspzleiGaDTs4ryZiv6uJ27uyAcEyPDmojiHf5BIQCLeIdLl5zyVPb0juHC4ZYxoU43Y0mYBiSPNZh9U2C6bd8sZxqt0",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -20.2979423,
                    "lng": 44.2694266
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.2965964197085,
                        "lng": 44.2707963302915
                    },
              

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAN5a9oLZ07tfjS1LN5Kp0ellrcRuvMM7kGA41MQLWe04n31eXR7_xsIedW7GqL2bayZkFpPL91lBaFSy-oFLejCGwjfjKXxqpoWgJICeNkJAfZr-jighVFcg7zjJ-Ktd1OWIP--AE__34xRLmTSdEdFwLvX2rSvGoIW6XVWIadKnU0mmUy_dTKYlAz2d3bzLylW6UmPabe0buL4ytdtsTMosbKlxFgFVjIy27ER-c-3kMj9Wv6K-3ysdoY__gyQjQaxBNwLQp6JkPO0GoZLVkDy1IBhWAFt-SCTaGxCri8iH0hXDcuj1tXyI747f1N8NX11HAqVMxE2qNsb05qijKZ9mmaNI_2r-HcoacwsoTkHVBdYnmSJ9zpL8g3W66BqVjxIQHbvoQPO92C2aR-oouwqOqRoU3OABuyBl_AO-ilrODmS5tyAPRpo",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -33.30587549999999,
                    "lng": -66.3451602
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.3045444697085,
                        "lng": -66.34380691970848
                    },
                    "southwest": {
                        "lat": -33.3072424302915,
                        "lng": -66

In [17]:
# The CitiesPy module returned the Adrar region of Mauritania instead of a city. It is in the Sahara dessert.
# There is no hotel within the search range because the coordinates are in the middle of a desert away from a town with lodging
hotel_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum Temperature,Wind Speed,Hotel Name
180,Adrar,0,MR,1587327241,16,20.50,-10.07,79.21,7.43,
329,Morondava,0,MG,1587327415,73,-20.28,44.28,78.60,7.45,La Case Bambou
409,San Luis,0,AR,1587327333,29,-33.30,-66.34,76.96,6.17,Huarpes Hotel


In [18]:
narrowed_city_df=hotel_df.copy()

In [19]:
# This removes the row that does not have a hotel. The reason there is no hotel is above.
narrowed_city_df=narrowed_city_df.drop(narrowed_city_df.index[0])
narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum Temperature,Wind Speed,Hotel Name
329,Morondava,0,MG,1587327415,73,-20.28,44.28,78.60,7.45,La Case Bambou
409,San Luis,0,AR,1587327333,29,-33.30,-66.34,76.96,6.17,Huarpes Hotel


In [20]:
narrowed_city_df.rename(columns = {'Latitude':"Lat", 'Longitude':"Lng"}, inplace = True)

In [21]:
narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Maximum Temperature,Wind Speed,Hotel Name
329,Morondava,0,MG,1587327415,73,-20.28,44.28,78.60,7.45,La Case Bambou
409,San Luis,0,AR,1587327333,29,-33.30,-66.34,76.96,6.17,Huarpes Hotel


In [31]:
# NOTE: Do not change any of the code in this cell
# I had to change the locations like at the end to the narrowed_city_df data frame for this to run

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [32]:
#info box setup for hotel names
hotel_name_lt=narrowed_city_df["Hotel Name"].tolist()

In [33]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=[f"Hotel Name: {element}" for element in hotel_name_lt])
# Add the layer to the map
fig.add_layer(markers)
fig
# Display Map

Figure(layout=FigureLayout(height='420px'))